In [2]:
# Author: Dýrmundur Helgi R. Óskarsson
# Date: 20.9.2023
# Project: 05 Classification
# Acknowledgements: Einar Óskar & Torfi Tímóteus
#


from tools import load_iris, split_train_test

import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import multivariate_normal

In [3]:
# Section 1.1

def mean_of_class(
    features: np.ndarray,
    targets: np.ndarray,
    selected_class: int
) -> np.ndarray:
    '''
    Estimate the mean of a selected class given all features
    and targets in a dataset
    '''
    class_mean = np.mean(features[targets == selected_class], axis=0)
    
    return class_mean
   

#features, targets, classes = load_iris()
#(train_features, train_targets), (test_features, test_targets) = split_train_test(features, targets, train_ratio = 0.6)

#print(mean_of_class(train_features, train_targets, 0))

[4.971875 3.3875   1.434375 0.246875]


In [4]:
# Section 1.2

def covar_of_class(
    features: np.ndarray,
    targets: np.ndarray,
    selected_class: int
) -> np.ndarray:
    '''
    Estimate the covariance of a selected class given all
    features and targets in a dataset
    '''
    class_cov = np.cov(features[targets == selected_class], rowvar=False)
    
    return class_cov

#print(covar_of_class(train_features, train_targets, 0))

[[0.13563508 0.11673387 0.0174496  0.01523185]
 [0.11673387 0.17016129 0.01141129 0.01479839]
 [0.0174496  0.01141129 0.03265121 0.00478831]
 [0.01523185 0.01479839 0.00478831 0.00966734]]


In [5]:
# Section 1.3

def likelihood_of_class(
    feature: np.ndarray,
    class_mean: np.ndarray,
    class_covar: np.ndarray
) -> float:
    '''
    Estimate the likelihood that a sample is drawn
    from a multivariate normal distribution, given the mean
    and covariance of the distribution.
    '''
    mvn = multivariate_normal(class_mean, class_covar)
    likelihood = mvn.pdf(feature)
    
    return likelihood
    
#class_mean = mean_of_class(train_features, train_targets, 0)
#class_cov = covar_of_class(train_features, train_targets, 0)
#print(likelihood_of_class(test_features[0, :], class_mean, class_cov))

2.5183032809117973


In [6]:
def maximum_likelihood(
    train_features: np.ndarray,
    train_targets: np.ndarray,
    test_features: np.ndarray,
    classes: list
) -> np.ndarray:
    '''
    Calculate the maximum likelihood for each test point in
    test_features by first estimating the mean and covariance
    of all classes over the training set.

    You should return
    a [test_features.shape[0] x len(classes)] shaped numpy
    array
    '''
    means, covs = [], []
    for class_label in classes:
        class_mean = mean_of_class(train_features, train_targets, class_label)
        class_cov = covar_of_class(train_features, train_targets, class_label)
        
        means.append(class_mean), covs.append(class_cov)
    likelihoods = []
    for i in range(test_features.shape[0]):
        likelihood = []
        for j in range(len(classes)):
            likelihood.append(likelihood_of_class(train_features[i], means[j], covs[j]))
        likelihoods.append(likelihood)
    return np.array(likelihoods)

#print(maximum_likelihood(train_features, train_targets, test_features, classes))

[[7.87608368e-116 5.15389915e-002 1.58685993e-001]
 [3.20814672e-085 1.73314212e+000 5.37011170e-004]
 [8.36399009e-052 5.38397718e-001 3.70323661e-005]
 [7.50380637e+000 5.00557693e-026 2.70558330e-036]
 [4.14973303e-233 6.10806507e-008 1.37023660e-001]
 [7.38275800e-201 1.62783390e-007 1.26101112e+000]
 [9.23046883e-066 2.38147431e+000 6.08721596e-003]
 [1.56841031e-001 1.15485566e-027 5.45846749e-037]
 [9.13355510e-085 2.13350936e-001 4.28732288e-003]
 [1.44476128e-001 2.68430751e-020 2.97226225e-027]
 [1.28895416e-064 7.05249278e-001 2.57877366e-004]
 [1.47788578e-091 2.24531842e+000 2.61812533e-002]
 [1.96226171e-146 7.85084293e-004 1.22200436e+000]
 [3.96659593e+000 3.96078730e-020 1.25327138e-028]
 [6.05800285e-089 7.99447306e-001 3.08665358e-002]
 [4.89829838e-139 2.53660019e-004 7.26346852e-001]
 [6.44328126e-001 5.58592186e-036 4.87549765e-049]
 [1.09729849e+001 1.19724133e-025 2.22561472e-035]
 [1.37118361e-189 9.80458836e-005 4.59105828e-001]
 [1.11349489e-095 1.74753149e+0

In [7]:
# Section 1.5

def predict(likelihoods: np.ndarray):
    '''
    Given an array of shape [num_datapoints x num_classes]
    make a prediction for each datapoint by choosing the
    highest likelihood.

    You should return a [likelihoods.shape[0]] shaped numpy
    array of predictions, e.g. [0, 1, 0, ..., 1, 2]
    '''
    return np.argmax(likelihoods, axis=1)

#likelihoods = maximum_likelihood(train_features, train_targets, test_features, classes)
#print(predict(likelihoods))

[2 1 1 0 2 2 1 0 1 0 1 1 2 0 1 2 0 0 2 1 1 1 1 1 1 0 2 0 0 0 1 2 0 1 1 0 1
 1 1 2 1 0 1 0 0 2 0 0 2 0 0 2 0 2 0 1 2 0 0]


In [10]:
# Section 2.1

def maximum_aposteriori(
    train_features: np.ndarray,
    train_targets: np.ndarray,
    test_features: np.ndarray,
    classes: list
) -> np.ndarray:
    '''
    Calculate the maximum a posteriori for each test point in
    test_features by first estimating the mean and covariance
    of all classes over the training set.

    You should return
    a [test_features.shape[0] x len(classes)] shaped numpy
    array
    '''
    means, covs, priors = [], [], []
    length_features = len(train_features)
    for class_label in classes:
        class_mean = mean_of_class(train_features, train_targets, class_label)
        class_cov = covar_of_class(train_features, train_targets, class_label)
        prior = len(train_features[train_targets == class_label]) / length_features
        
        means.append(class_mean), covs.append(class_cov), priors.append(prior)
    likelihoods = []
    for i in range(test_features.shape[0]):
        likelihood = []
        for j in range(len(classes)):
            likelihood.append((likelihood_of_class(train_features[i], means[j], covs[j])) * priors[j])
        likelihoods.append(likelihood)
    return np.array(likelihoods)

#posteriors = maximum_aposteriori(train_features, train_targets, test_features, classes)
#print(posteriors)

#predictions = predict(posteriors)
#print(predictions)

[2 1 1 0 2 2 1 0 1 0 1 1 2 0 1 2 0 0 2 1 1 1 1 1 1 0 2 0 0 0 1 2 0 1 1 0 1
 1 1 2 1 0 1 0 0 2 0 0 2 0 0 2 0 2 0 1 2 0 0]


In [31]:
# Section 2.2

#predictions = predict(likelihoods)
#predictions = predict(posteriors)

#print(train_targets[0:len(predictions)])
#count=0
#for i, tt in enumerate(train_targets[0:len(predictions)]):
#    if tt==predictions[i]:
#        count+=1
#print(count/len(predictions))

#length = len(classes)
#confusion_matrix = np.zeros((length, length), dtype='int')

#for i in range(len(predictions)):
#    correct = train_targets[i]
#    guess = predictions[i]
#    confusion_matrix[guess][correct] += 1
    
#print(confusion_matrix)